# PJT_01

## 영화진흥위원회 오픈 API(주간/주말 박스오피스 데이터)
 - 요청 조건
    1. 주간(월~일)까지 기간의 데이터를 조회합니다.
    2. 조회 기간은 총 50주이며, 기준일(마지막 일자)은 2019년 7월 13일입니다.
    3. 다양성 영화/상업 영화를 모두 포함하여야 합니다.
    4. 한국/외국 영화를 모두 포함하여야 합니다.
    5. 모든 상영지역을 포함하여야 합니다.


 - 결과
    1. 수집된 데이터에서 영화 `대표코드` , `영화명` , `해당일 누적관객수` 를 기록합니다.
    2. 해당일 누적관객수 는 중복시 최신 정보를 반영하여야 합니다.
        - 예) 영화 엄복동이 20190713 기준 50,000명이고, 20190106 기준 5,000명이면 50,000명이 저장되어야 합니다.
    3. 해당 결과를 boxoffice.csv 에 저장합니다.

### API 사용하여 데이터 불러오기
API 문서: http://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do

---

문서를 읽어보니 **기본 요청 URL** 이 이렇게 생겼다. 이 URL 로 데이터를 불러올 수 있다고 한다.

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json

편의를 위해 앞으로 위 url 을 **기본 요청 URL** 로 표기하겠다.

링크를 클릭해서 들어가보니 아래와 같은 응답이 왔다. json 타입의 파일로 응답을 주는것으로 보인다.


```json
{
  "faultInfo": {
    "message": "invalidKey",
    "errorCode": "320010"
  }
}
```

`invalidKey` 라는 메세지가 왔다. API 를 사용하기 위한 key 값을 입력하지 않은 것 같다.

API 문서를 조금 더 아래로 내려 보니 다음과 같은 테이블이 있다.


| 요청 변수   | 값           | 설명                                                         |
| ----------- | ------------ | ------------------------------------------------------------ |
| key         | **문자열(필수)** | 발급받은키 값을 입력합니다.                                  |
| targetDt    | **문자열(필수)** | 조회하고자 하는 날짜를 yyyymmdd 형식으로 입력합니다.         |
| itemPerPage | 문자열       | 결과 ROW 의 개수를 지정합니다.(default : “10”, 최대 : “10“)  |
| weekGb      | 문자열       | 주간/주말/주중을 선택 입력합니다<br/>“0” : 주간 (월~일)<br/>“1” : 주말 (금~일) (default)<br/>“2” : 주중 (월~목) |


`key` 뿐만 아니라 `targetDt` 값도 필수로 넘겨줘야 하는 것 같다.

페이지 가장 아래 응답 예시의 요청 URL 을 확인해보니 **기본 요청 URL** 뒤에 `?` 캐릭터 이후로 요청 변수 값을 할당하는 것으로 확인된다. 그리고 다음 요청 변수를 적을 때는 `&` 캐릭터로 구분이 되고 있다.

<**기본 요청 URL**>?<span style="color: blue">**key**</span>=430156241533f1d058c603178cc3ca0e&<span style="color: blue">**targetDt**</span>=20120101

---

그럼 발급받은 키로 위와 같은 URL 을 만들어서 요청을 보내보자.

In [ ]:
BASE_URL = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json'

import requests
import pprint from  
API_KEY =
targetDt
API_URL = f'{BASE_URL} + 

추가로 해당 API는 기본 요청이 `주말(금~일)` 데이터 호출이 default 값이므로 `주간(월~일)` 데이터를 가지고 올 수 있도록 추가 요청 변수를 설정해야겠다.

 `weekGb=0` ( 어떤 변수를 줘야할 지 모른다면 API 문서 내 요청 변수가 정의되어 있는 테이블을 확인 해보자 )

---

### 응답 결과에서 원하는 데이터만 추출

영화 데이터 응답받기는 성공했다.

이제 우리가 원하는 특정 데이터만 꺼내서 저장 해보자.

```json
{'boxOfficeResult': {'boxofficeType': '주말 박스오피스',
                     'showRange': '20190712~20190714',
                     'weeklyBoxOfficeList': [{'audiAcc': '6685136',
                                              'audiChange': '-54.4',
                                              'audiCnt': '1302522',
                                              'audiInten': '-1555300',
                                              'movieCd': '20196309',
                                              'movieNm': '스파이더맨: 파 프롬 홈',
                                              ...
```

우리가 원하는 데이터는 API 를 통해 불러온 영화 목록이며 `weeklyBoxOfficeList` 의 value 값으로 담겨져 있는 list 이다.

list 의 item 으로는 영화 데이터가 dict 형태로 저장되어 있으며 그 중에서도 명세서에 적혀있는 `대표코드` , `영화명` , `해당일 누적관객수` 값을 추출해야 한다.

API 문서 중 응답구조를 읽어보면 dict 안에 있는 `movieCd`, `movieNm` 그리고 `audiAcc` 가 우리가 원하는 값임을 확인할 수 있다.

그럼 전체 응답 받은 데이터에서 해당 정보만 추출해서 저장 해보자

In [13]:
import requests
import csv
from pprint import pprint
from datetime import datetime, timedelta
from decouple import config
BASE_URL = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json'
API_KEY = 'ad91a841408dc2a6520a60cf883ef8d1'
targetDt = datetime(2019, 7, 13) - timedelta(weeks=1)
targetDt = targetDt.strftime('%Y%m%d') 
API_URL = f'{BASE_URL}?key={API_KEY}&targetDt={targetDt}&weekGb=0'
#json->dict형식으로 변환
response = requests.get(API_URL).json()

movie_list = {}

# 이전 기준일에 대한 요청코드가 50주의 데이터를 요청하는 코드로 바꿔보자
for weeks in range(5, 0, -1) :
    # 0 ~ 50 주를 기준일에서 뺴면서 반복
    print(weeks)
    targetDt_raw = datetime(2019, 7, 13) - timedelta(weeks=weeks)
    targetDt = targetDt_raw.strftime('%Y%m%d') #yyyymmdd 형식으로 변환
    
    #50주 이전까지 새로운 요청을 보낸다.
    API_URL = f'{BASE_URL}?key={API_KEY}&targetDt={targetDt}&weekGb=0'
    response = requests.get(API_URL).json()
#응답받은 데이터에서 weeklyBoxOfficeList로 접근
    weekly_box_office_list = response.get('boxOfficeResult').get('weeklyBoxOfficeList')

    for movie in weekly_box_office_list :
        #영화의 대표코드는 해당 영화의 유일한 값이므로 이 값을 key값으로 하여 우리가 원하는 
        #영화의 데이터를 'movie_list'에 저장한다.
        code = movie.get('movieCd')
        #50주 동안의 주간 박스오피스 목록이기 때문에 중복되는 목록이 있을 것이다.
        #해당 영화의 코드 값이 movie_list의 키값으로 존재하는지 확인을 하고 해당 영화가 저장되지 않았을 경우에만 저장한다.

        movie_list[code] = {
            'movieCd' : movie.get('movieCd'),
            'movieNm' : movie.get('movieNm'),
            'audiAcc' : movie.get('audiAcc'),
        }
pprint(movie_list)

5
4
3
2
1
{'19880001': {'audiAcc': '142372', 'movieCd': '19880001', 'movieNm': '이웃집 토토로'},
 '20182387': {'audiAcc': '860921',
              'movieCd': '20182387',
              'movieNm': '엑스맨: 다크 피닉스'},
 '20182585': {'audiAcc': '199009', 'movieCd': '20182585', 'movieNm': '비스트'},
 '20183782': {'audiAcc': '9774306', 'movieCd': '20183782', 'movieNm': '기생충'},
 '20183867': {'audiAcc': '9222631', 'movieCd': '20183867', 'movieNm': '알라딘'},
 '20184047': {'audiAcc': '2783699',
              'movieCd': '20184047',
              'movieNm': '토이 스토리 4'},
 '20184865': {'audiAcc': '352862',
              'movieCd': '20184865',
              'movieNm': '고질라: 킹 오브 몬스터'},
 '20184889': {'audiAcc': '13887813',
              'movieCd': '20184889',
              'movieNm': '어벤져스: 엔드게임'},
 '20188144': {'audiAcc': '133522', 'movieCd': '20188144', 'movieNm': '0.0MHz'},
 '20188941': {'audiAcc': '3346654', 'movieCd': '20188941', 'movieNm': '악인전'},
 '20190273': {'audiAcc': '228365',
              'movieCd': '2019

---

### 50주 동안의 데이터 불러와서 저장하기

명세서 기준일이 2019.07.13 이고 해당 날짜에서부터 50 주 이전까지의 데이터를 수집해야한다.

다행히 파이썬에서는 `기준일 - 1주` 와 같은 연산이 된다.

In [1]:
from datetime import datetime, timedelta


targetDt = datetime(2019, 7, 13) - timedelta(weeks=12)
targetDt = targetDt.strftime('%Y%m%d')
print(targetDt)

2019-04-20 00:00:00


그럼 CSV 파일로 저장하기 전에 `기준일 - 1주` 부터 `기준일 - 50주` 까지의 데이터를 모두 수집하면서 변수에 전부 저장해보자.

---

### 저장된 영화 목록을 csv 파일 형태로 저장하기

파이썬으로 csv 형식의 파일을 저장하는 방법으로는 여러가지가 있었다고 배웠다.

**그 중에서도 csv 의 field 값을 key 값으로 하는 dict 를 저장하는 방법을 사용하겠다.**

우리는 다음과 같은 형태의 dict 를 가지고 있으며 각 key 값은 우리가 저장하고자 하는 데이터의 field 값과 같다.

```python
{'20183782': {'audiAcc': '9919835', 'movieCd': '20183782', 'movieNm': '기생충'},
 '20183867': {'audiAcc': '10161231', 'movieCd': '20183867', 'movieNm': '알라딘'},
 '20184047': {'audiAcc': '3151060',
              'movieCd': '20184047',
              'movieNm': '토이 스토리 4'},
 ...
}
 ```

In [16]:
import csv
with open('boxoffice.csv', 'w', encoding='utf=8', newline='') as f:
    #우리가 저장하고자 하는 필드이름들을 정의
    fieldnames = ('moiveCd', 'movieNm', 'audiAcc')
    #csv 를 작성해주는 객체를 생성, 위에서 정의한 필드이름을 옵션으로 넘겨줌
    writer = csv.DicWriter(f, fieldnames = fieldnames)
    
    #csv 파일 최상단에 필드이름을 작성
    writer.writeheader()
    
    for movie in movie_list.values() :
        print(movie)
        print()

AttributeError: module 'csv' has no attribute 'DicWriter'